In [1]:
import gzip
import math
import random
from collections import defaultdict
import pandas as pd
import ast
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
path = "goodreads_reviews_dedup.json.gz"
f_reviews = gzip.open(path, 'rt', encoding="utf8")

heade_reviews = f_reviews.readlines()

In [3]:
heade_reviews[0]

'{"user_id": "8842281e1d1347389f2ab93d60773d4d", "book_id": "24375664", "review_id": "5cd416f3efc3f944fce4ce2db2290d5e", "rating": 5, "review_text": "Mind blowingly cool. Best science fiction I\'ve read in some time. I just loved all the descriptions of the society of the future - how they lived in trees, the notion of owning property or even getting married was gone. How every surface was a screen. \\n The undulations of how society responds to the Trisolaran threat seem surprising to me. Maybe its more the Chinese perspective, but I wouldn\'t have thought the ETO would exist in book 1, and I wouldn\'t have thought people would get so over-confident in our primitive fleet\'s chances given you have to think that with superior science they would have weapons - and defenses - that would just be as rifles to arrows once were. \\n But the moment when Luo Ji won as a wallfacer was just too cool. I may have actually done a fist pump. Though by the way, if the Dark Forest theory is right - an

In [4]:
len(heade_reviews)

15739967

### Considering 1M reviews

In [ ]:
header_reviews = heade_reviews
usersPerItem = defaultdict(set) # Maps an item to the users who rated it
itemsPerUser = defaultdict(set) # Maps a user to the items that they rated
# itemNames = {}
ratingDict = {} # To retrieve a rating for a specific user/item pair
for review_info in tqdm(header_reviews[:100000]):
    review = ast.literal_eval(review_info)
    user_id = review['user_id']
    book_id = review['book_id']
    rating = review['rating']
    user,item = user_id, book_id
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    ratingDict[(user,item)] = rating
#     print(user_id, book_id, rating)
#     break

 78%|████████████████████████████████████████████████████████▍               | 78380/100000 [07:29<00:00, 22246.99it/s]

In [14]:
userAverages = {}
itemAverages = {}

for u in itemsPerUser:
    rs = [ratingDict[(u,i)] for i in itemsPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)


Jaccard

In [15]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

Cosine

In [16]:
def CosineSet(s1, s2):
    # Not a proper implementation, operates on sets so correct for interactions only
    numer = len(s1.intersection(s2))
    denom = math.sqrt(len(s1)) * math.sqrt(len(s2))
    if denom == 0:
        return 0
    return numer / denom

In [17]:
def Cosine(i1, i2):
    # Between two items
    inter = usersPerItem[i1].intersection(usersPerItem[i2])
    numer = 0
    denom1 = 0
    denom2 = 0
    for u in inter:
        numer += ratingDict[(u,i1)]*ratingDict[(u,i2)]
    for u in usersPerItem[i1]:
        denom1 += ratingDict[(u,i1)]**2
    for u in usersPerItem[i2]:
        denom2 += ratingDict[(u,i2)]**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

Pearson

In [18]:
def Pearson(i1, i2):
    # Between two items
    iBar1 = itemAverages[i1]
    iBar2 = itemAverages[i2]
    inter = usersPerItem[i1].intersection(usersPerItem[i2])
    numer = 0
    denom1 = 0
    denom2 = 0
    for u in inter:
        numer += (ratingDict[(u,i1)] - iBar1)*(ratingDict[(u,i2)] - iBar2)
    for u in inter: #usersPerItem[i1]:
        denom1 += (ratingDict[(u,i1)] - iBar1)**2
    #for u in usersPerItem[i2]:
        denom2 += (ratingDict[(u,i2)] - iBar2)**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

In [19]:
def mostSimilar(i, N):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        #sim = Pearson(i, i2) # Could use alternate similarity metrics straightforwardly
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:10]

In [20]:
query = ast.literal_eval(header_reviews[0])['book_id']
query

'24375664'

In [23]:
ms = mostSimilar(query, 10)
ms

[(0.6666666666666666, '6563914'),
 (0.6666666666666666, '18245960'),
 (0.3333333333333333, '98233'),
 (0.3333333333333333, '9793184'),
 (0.3333333333333333, '9624'),
 (0.3333333333333333, '9460786'),
 (0.3333333333333333, '9339'),
 (0.3333333333333333, '918095'),
 (0.3333333333333333, '912'),
 (0.3333333333333333, '897461')]

In [24]:
path = "goodreads_books.json.gz"
f_books = gzip.open(path, 'rt', encoding="utf8")

header_books = f_books.readlines()

In [25]:
len(header_books)

2360655

In [26]:
header_books[0]

'{"isbn": "0312853122", "text_reviews_count": "1", "series": [], "country_code": "US", "language_code": "", "popular_shelves": [{"count": "3", "name": "to-read"}, {"count": "1", "name": "p"}, {"count": "1", "name": "collection"}, {"count": "1", "name": "w-c-fields"}, {"count": "1", "name": "biography"}], "asin": "", "is_ebook": "false", "average_rating": "4.00", "kindle_asin": "", "similar_books": [], "description": "", "format": "Paperback", "link": "https://www.goodreads.com/book/show/5333265-w-c-fields", "authors": [{"author_id": "604031", "role": ""}], "publisher": "St. Martin\'s Press", "num_pages": "256", "publication_day": "1", "isbn13": "9780312853129", "publication_month": "9", "edition_information": "", "publication_year": "1984", "url": "https://www.goodreads.com/book/show/5333265-w-c-fields", "image_url": "https://images.gr-assets.com/books/1310220028m/5333265.jpg", "book_id": "5333265", "ratings_count": "3", "work_id": "5400751", "title": "W.C. Fields: A Life on Film", "ti

In [27]:
# book_info_dict = {}
# for book_info in tqdm(header_books):
#     info = ast.literal_eval(book_info)
# #     print(info)
#     book_info_dict[info['book_id']] = info['title']
    
# #     break

In [28]:
import json
f = open('book_info_dict.json')
book_info_dict = json.load(f)

In [29]:
book_info_dict = {int(k):v for k,v in book_info_dict.items()}

In [30]:
print("The similar books based on item-item recommendation:\n")
for tuple_val in ms:
    if int(tuple_val[1]) in list(book_info_dict.keys()):
        try:
            print(book_info_dict[int(tuple_val[1])])
        except Exception as e:
            print(e)

The similar books based on item-item recommendation:

The Diamond Age
The Three-Body Problem (Remembrance of Earth’s Past, #1)
Founders at Work: Stories of Startups' Early Days
The Lost Books of The Odyssey
The Martian Chronicles
The Clock of the Long Now: Time and Responsibility
Surfing the Himalayas: A Spiritual Adventure
The Religion (Tannhauser, #1)
Rich Dad, Poor Dad
Islandia


In [31]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

In [32]:
for review_info in tqdm(header_reviews[:1000000]):
    review = ast.literal_eval(review_info)
    user = review['user_id']
    book = review['book_id']
    reviewsPerUser[user].append(review)
    reviewsPerItem[item].append(review)

100%|██████████████████████████████████████████████████████████████████████| 1000000/1000000 [07:23<00:00, 2252.46it/s]


In [33]:
items = set(usersPerItem.keys())
users = set(itemsPerUser.keys())

In [34]:
# 1: Average cosine similarity between i and items in u's history
def rec1score(u, i, userHistory):
    if len(userHistory) == 0:
        return 0
    averageSim = []
    s1 = usersPerItem[i].difference(set([u]))
    for h in userHistory:
        s2 = usersPerItem[h].difference(set([u]))
        averageSim.append(Jaccard(s1,s2))
    averageSim = sum(averageSim)/len(averageSim)
    return averageSim

# 2: Jaccard similarity with most similar user who has consumed i
def rec2score(u, i, userHistory):
    bestSim = None
    for v in usersPerItem[i]:
        if u == v:
            continue
        sim = Jaccard(userHistory, itemsPerUser[v])
        if bestSim == None or sim > bestSim:
            bestSim = sim
    if bestSim == None:
        return 0
    return bestSim

# Generate a recommendation for a user based on a given scoring function
def rec(u, score):
    history = itemsPerUser[u]
    if len(history) > 5: # If the history is too long, just take a sample
        history = random.sample(history,5)
    bestItem = None
    bestScore = None
    for i in items:
        if i in itemsPerUser[u]: continue
        s = score(u, i, history)
        if bestItem == None or s > bestScore:
            bestItem = i
            bestScore = s

    return bestItem, bestScore

In [35]:
u = random.sample(users,1)[0]

C:\Users\pramo\AppData\Local\Temp/ipykernel_2376/1002506522.py:1: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  u = random.sample(users,1)[0]


In [36]:
u

'0c316fe9475f27445f79eda1c44d1eec'

In [37]:
out1 = rec(u, rec1score)
out1

('6748', 0.0)

In [38]:
def recTest(simFunction, nUserSamples):
    items = set(usersPerItem.keys())
    users = list(itemsPerUser.keys())
    
    better = 0
    worse = 0

    for u in random.sample(users, nUserSamples):
        itemsU = set(itemsPerUser[u])
        if len(itemsU) < 2:
            continue
        i = random.sample(itemsU, 1)[0]
        uWithheld = itemsU.difference(set([i]))
        j = random.sample(items,1)[0]

        si = simFunction(u,i,uWithheld)
        sj = simFunction(u,j,uWithheld)
        
        if si > sj:
            better += 1
        if sj > si:
            worse += 1
            
    print("Better than random " + str(better) + " times")
    print("Worse than random " + str(worse) + " times")

In [40]:
recTest(rec1score,500)

C:\Users\pramo\AppData\Local\Temp/ipykernel_2376/567367000.py:12: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  i = random.sample(itemsU, 1)[0]
C:\Users\pramo\AppData\Local\Temp/ipykernel_2376/567367000.py:14: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  j = random.sample(items,1)[0]


Better than random 148 times
Worse than random 59 times


#### (iii)

In [53]:
def predictRating1(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: continue
        ratings.append(d['rating'])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return sum(weightedRatings) / sum(similarities)
    else:
        return ratingMean

In [54]:
def predictRating2(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerItem[item]:
        u2 = d['user_id']
        if u2 == user: continue
        ratings.append(d['rating'])
        similarities.append(Jaccard(itemsPerUser[user],itemsPerUser[u2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return sum(weightedRatings) / sum(similarities)
    else:
        return ratingMean

In [55]:
def predictRating3(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: continue
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        return ratingMean

In [59]:
simPredictions1 = []
simPredictions2 = []
simPredictions3 = []
for review_info in tqdm(header_reviews[:1000000]):
    try:
        review = ast.literal_eval(review_info)
        user = review['user_id']
        book = review['book_id']
        simPredictions1.append(predictRating1(user, book))
        simPredictions2.append(predictRating2(user, book))
        simPredictions3.append(predictRating3(user, book))
    except:
        continue

100%|███████████████████████████████████████████████████████████████████████| 1000000/1000000 [43:47<00:00, 380.63it/s]


In [61]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [68]:
len(simPredictions1)

442707

In [67]:
labels = []
for review_info in tqdm(header_reviews[:1000000]):
    review = ast.literal_eval(review_info)
    labels.append(review['rating'])

100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:40<00:00, 24409.65it/s]


In [69]:
MSE(simPredictions1, labels)

2.0107457242950795

In [70]:
MSE(simPredictions2, labels)

1.9453432939230009

In [71]:
MSE(simPredictions3, labels)

2.4859194272991023